In [ ]:
! wget http://images.cocodataset.org/zips/train2014.zip
! wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
! sudo apt-get install unzip
! unzip train2014.zip -d coco
! unzip annotations_trainval2014.zip -d coco

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexcalabrese/fdl_project/main/requirements.txt


# Image Captioning

In [2]:
import pandas as pd
import os
import json


class DatasetLoader:

    def __init__(self, folder_name: str = "coco"):
        self.data = self.load_data(folder_name)

    def load_data(self, folder_name: str = "coco"):
        print("Loading data..")
        # Set the data folder
        data_folder = os.path.join(folder_name)

        # Create the directory if it doesn't exist
        if not os.path.exists(data_folder):
            os.makedirs(data_folder)

        # Path to the captions JSON file
        filename = os.path.join(data_folder, "annotations", "captions_train2014.json")

        # Read and parse the JSON file
        with open(filename, "r") as json_file:
            data = json.load(json_file)

        print("Data loaded succesfully!")

        return data

    def get_data_dict(self):
        return self.data

    def get_df_annotations(self):
        return pd.DataFrame(self.data["annotations"])

    def get_df_images(self):
        return pd.DataFrame(self.data["images"])


## Setup

In [3]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import re
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization

keras.utils.set_random_seed(111)

## Download the dataset

We will be using the Flickr8K dataset for this tutorial. This dataset comprises over
8,000 images, that are each paired with five different captions.

In [4]:
# Desired image dimensions
IMAGE_SIZE = (299, 299)

# Vocabulary size
VOCAB_SIZE = 10000

# Fixed length allowed for any sequence
SEQ_LENGTH = 25

# Dimension for the image embeddings and token embeddings
EMBED_DIM = 512

# Per-layer units in the feed-forward network
FF_DIM = 512

# Other training parameters
BATCH_SIZE = 256
EPOCHS = 15
AUTOTUNE = tf.data.AUTOTUNE

## Preparing the dataset

In [7]:
import json
import os
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.xception import preprocess_input
# from fdl_project.classes.dataset_loader import DatasetLoader

# Load the dataset
dataset = DatasetLoader().load_data(folder_name=os.path.join(os.getcwd(), "coco"))

Loading data..
Data loaded succesfully!
Loading data..
Data loaded succesfully!


In [8]:
def load_captions_data(dataset):
    text_annotations = []

    # Create a mapping from image_id to file_name
    image_id_to_file_name = {image["id"]: image["file_name"] for image in dataset["images"]}

    # Create a mapping from image_id to captions
    image_id_to_captions = {}
    for annotation in dataset["annotations"]:
        image_id = annotation["image_id"]
        caption = "<start> " + annotation["caption"] + " <end>"

        if image_id not in image_id_to_captions:
            image_id_to_captions[image_id] = []

        image_id_to_captions[image_id].append(caption)
        text_annotations.append(caption)

    # Create a combined dictionary with image file names and their corresponding captions
    file_name_to_captions = {
        image_id_to_file_name[image_id]: captions
        for image_id, captions in image_id_to_captions.items()
    }

    return file_name_to_captions, text_annotations


def train_val_split(caption_data, train_size=0.8, sample_size=0.001, shuffle=True):
    all_images = list(caption_data.keys())

    if shuffle:
        np.random.shuffle(all_images)

    # Calculate the number of samples based on the sample_size percentage
    num_samples = int(len(caption_data) * sample_size)

    # Reduce the dataset to the specified sample size
    all_images = all_images[:num_samples]

    train_size = int(len(all_images) * train_size)

    training_data = {
        img_name: caption_data[img_name] for img_name in all_images[:train_size]
    }
    validation_data = {
        img_name: caption_data[img_name] for img_name in all_images[train_size:]
    }

    return training_data, validation_data

# Load the dataset
captions_mapping, text_data = load_captions_data(dataset)

# Split the dataset into training and validation sets
train_data, valid_data = train_val_split(captions_mapping, sample_size=1.0)
print("Number of training samples: ", len(train_data))
print("Number of validation samples: ", len(valid_data))

Number of training samples:  66226
Number of validation samples:  16557


In [9]:
# captions_mapping

## Vectorizing the text data

We'll use the `TextVectorization` layer to vectorize the text data,
that is to say, to turn the
original strings into integer sequences where each integer represents the index of
a word in a vocabulary. We will use a custom string standardization scheme
(strip punctuation characters except `<` and `>`) and the default
splitting scheme (split on whitespace).

In [10]:

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


strip_chars = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
strip_chars = strip_chars.replace("<", "")
strip_chars = strip_chars.replace(">", "")

vectorization = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=SEQ_LENGTH,
    standardize=custom_standardization,
)
vectorization.adapt(text_data)

# Data augmentation for image data
image_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomContrast(0.3),
    ]
)


## Building a `tf.data.Dataset` pipeline for training

We will generate pairs of images and corresponding captions using a `tf.data.Dataset` object.
The pipeline consists of two steps:

1. Read the image from the disk
2. Tokenize all the five captions corresponding to the image

In [11]:
# append coco/train2014/ to all the keys of the dictionary

train_data = {
    os.path.join("coco/train2014/", key): value for key, value in train_data.items()
}

valid_data = {
    os.path.join("coco/train2014/", key): value for key, value in valid_data.items()
}

In [12]:
# train_data["coco/train2014/COCO_train2014_000000254362.jpg"]

In [13]:
# check in train_data if there are more than 5 values
# remove the 6th value and keep the first 5 values

for key, value in train_data.items():
    if len(value) > 5:
        # print(key, len(value))
        train_data[key] = value[:5]

for key, value in valid_data.items():
    if len(value) > 5:
        # print(key, len(value))
        valid_data[key] = value[:5]

In [14]:
def decode_and_resize(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, IMAGE_SIZE)
    img = tf.image.convert_image_dtype(img, tf.float32)
    return img


def process_input(img_path, captions):
    print("PROCESS_INPUT: LEN DATASET", len(dataset))
    return decode_and_resize(img_path), vectorization(captions)


def make_dataset(images, captions):
    dataset = tf.data.Dataset.from_tensor_slices((images, tf.ragged.constant((captions))))
    dataset = dataset.shuffle(BATCH_SIZE * 8)
    dataset = dataset.map(process_input, num_parallel_calls=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

    return dataset


# Pass the list of images and the list of corresponding captions
train_dataset = make_dataset(list(train_data.keys()), list(train_data.values()))

valid_dataset = make_dataset(list(valid_data.keys()), list(valid_data.values()))

PROCESS_INPUT: LEN DATASET 4
PROCESS_INPUT: LEN DATASET 4


## Building the model

Our image captioning architecture consists of three models:

1. A CNN: used to extract the image features
2. A TransformerEncoder: The extracted image features are then passed to a Transformer
                    based encoder that generates a new representation of the inputs
3. A TransformerDecoder: This model takes the encoder output and the text data
                    (sequences) as inputs and tries to learn to generate the caption.

In [15]:
def get_cnn_model():
    base_model = efficientnet.EfficientNetB0(
        input_shape=(*IMAGE_SIZE, 3),
        include_top=False,
        weights="imagenet",
    )
    # We freeze our feature extractor
    base_model.trainable = False
    base_model_out = base_model.output
    base_model_out = layers.Reshape((-1, base_model_out.shape[-1]))(base_model_out)
    cnn_model = keras.models.Model(base_model.input, base_model_out)
    return cnn_model


class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.0
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.dense_1 = layers.Dense(embed_dim, activation="relu")

    def call(self, inputs, training, mask=None):
        inputs = self.layernorm_1(inputs)
        inputs = self.dense_1(inputs)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=None,
            training=training,
        )
        out_1 = self.layernorm_2(inputs + attention_output_1)
        return out_1


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.embed_scale = tf.math.sqrt(tf.cast(embed_dim, tf.float32))

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_tokens = embedded_tokens * self.embed_scale
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, ff_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim, dropout=0.1
        )
        self.ffn_layer_1 = layers.Dense(ff_dim, activation="relu")
        self.ffn_layer_2 = layers.Dense(embed_dim)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()

        self.embedding = PositionalEmbedding(
            embed_dim=EMBED_DIM,
            sequence_length=SEQ_LENGTH,
            vocab_size=VOCAB_SIZE,
        )
        self.out = layers.Dense(VOCAB_SIZE, activation="softmax")

        self.dropout_1 = layers.Dropout(0.3)
        self.dropout_2 = layers.Dropout(0.5)
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, training, mask=None):
        inputs = self.embedding(inputs)
        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.cast(mask[:, :, tf.newaxis], dtype=tf.int32)
            combined_mask = tf.cast(mask[:, tf.newaxis, :], dtype=tf.int32)
            combined_mask = tf.minimum(combined_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=combined_mask,
            training=training,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
            training=training,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        ffn_out = self.ffn_layer_1(out_2)
        ffn_out = self.dropout_1(ffn_out, training=training)
        ffn_out = self.ffn_layer_2(ffn_out)

        ffn_out = self.layernorm_3(ffn_out + out_2, training=training)
        ffn_out = self.dropout_2(ffn_out, training=training)
        preds = self.out(ffn_out)
        return preds

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.constant([1, 1], dtype=tf.int32),
            ],
            axis=0,
        )
        return tf.tile(mask, mult)


class ImageCaptioningModel(keras.Model):
    def __init__(
        self,
        cnn_model,
        encoder,
        decoder,
        num_captions_per_image=5,
        image_aug=None,
    ):
        super().__init__()
        self.cnn_model = cnn_model
        self.encoder = encoder
        self.decoder = decoder
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.acc_tracker = keras.metrics.Mean(name="accuracy")
        self.num_captions_per_image = num_captions_per_image
        self.image_aug = image_aug

    def calculate_loss(self, y_true, y_pred, mask):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(mask, dtype=loss.dtype)
        loss *= mask
        return tf.reduce_sum(loss) / tf.reduce_sum(mask)

    def calculate_accuracy(self, y_true, y_pred, mask):
        accuracy = tf.equal(y_true, tf.argmax(y_pred, axis=2))
        accuracy = tf.math.logical_and(mask, accuracy)
        accuracy = tf.cast(accuracy, dtype=tf.float32)
        mask = tf.cast(mask, dtype=tf.float32)
        return tf.reduce_sum(accuracy) / tf.reduce_sum(mask)

    def _compute_caption_loss_and_acc(self, img_embed, batch_seq, training=True):
        encoder_out = self.encoder(img_embed, training=training)
        batch_seq_inp = batch_seq[:, :-1]
        batch_seq_true = batch_seq[:, 1:]
        mask = tf.math.not_equal(batch_seq_true, 0)
        batch_seq_pred = self.decoder(
            batch_seq_inp, encoder_out, training=training, mask=mask
        )
        loss = self.calculate_loss(batch_seq_true, batch_seq_pred, mask)
        acc = self.calculate_accuracy(batch_seq_true, batch_seq_pred, mask)
        return loss, acc

    def train_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        if self.image_aug:
            batch_img = self.image_aug(batch_img)

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            with tf.GradientTape() as tape:
                loss, acc = self._compute_caption_loss_and_acc(
                    img_embed, batch_seq[:, i, :], training=True
                )

                # 3. Update loss and accuracy
                batch_loss += loss
                batch_acc += acc

            # 4. Get the list of all the trainable weights
            train_vars = (
                self.encoder.trainable_variables + self.decoder.trainable_variables
            )

            # 5. Get the gradients
            grads = tape.gradient(loss, train_vars)

            # 6. Update the trainable weights
            self.optimizer.apply_gradients(zip(grads, train_vars))

        # 7. Update the trackers
        batch_acc /= float(self.num_captions_per_image)
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 8. Return the loss and accuracy values
        return {
            "loss": self.loss_tracker.result(),
            "acc": self.acc_tracker.result(),
        }

    def test_step(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_loss = 0
        batch_acc = 0

        # 1. Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # 2. Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and compute the loss as well as accuracy
        # for each caption.
        for i in range(self.num_captions_per_image):
            loss, acc = self._compute_caption_loss_and_acc(
                img_embed, batch_seq[:, i, :], training=False
            )

            # 3. Update batch loss and batch accuracy
            batch_loss += loss
            batch_acc += acc

        batch_acc /= float(self.num_captions_per_image)

        # 4. Update the trackers
        self.loss_tracker.update_state(batch_loss)
        self.acc_tracker.update_state(batch_acc)

        # 5. Return the loss and accuracy values
        return {
            "loss": self.loss_tracker.result(),
            "acc": self.acc_tracker.result(),
        }

    def predict(self, batch_data):
        batch_img, batch_seq = batch_data
        batch_pred = []

        # Get image embeddings
        img_embed = self.cnn_model(batch_img)

        # Pass each of the five captions one by one to the decoder
        # along with the encoder outputs and generate the predicted captions
        for i in range(self.num_captions_per_image):
            pred = self.decoder.predict(
                [img_embed, batch_seq[:, i, :]], batch_size=len(batch_img)
            )
            batch_pred.append(pred)

        # Return the predicted captions
        return batch_pred

    # Example usage:
    # batch_data = (batch_img, batch_seq)
    # predicted_captions = caption_model.predict(batch_data)
        # called automatically.
        return [self.loss_tracker, self.acc_tracker]


cnn_model = get_cnn_model()
encoder = TransformerEncoderBlock(embed_dim=EMBED_DIM, dense_dim=FF_DIM, num_heads=1)
decoder = TransformerDecoderBlock(embed_dim=EMBED_DIM, ff_dim=FF_DIM, num_heads=2)
caption_model = ImageCaptioningModel(
    cnn_model=cnn_model,
    encoder=encoder,
    decoder=decoder,
    image_aug=image_augmentation,
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Model training

In [23]:
try:
    from google.colab import drive
    import datetime
    drive.mount('/content/gdrive')

    # Create a directory in Google Drive to store models
    !mkdir /content/gdrive/My\ Drive/Colab_Models
    loss_model_path = "/content/gdrive/My Drive/Colab_Models/loss_model_transformer_efficientnet_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + ".txt"
    checkpoint_path = "/content/gdrive/My Drive/Colab_Models/cp_model_transformer_efficientnet_" + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + ".weights.h5"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    open(loss_model_path, 'w').close()
except:
    loss_model_path = "loss_model_from_scratch_LSTM_CNN.txt"
    checkpoint_path = "cp_model_from_scratch_LSTM_CNN.weights.h5"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    open(loss_model_path, 'w').close()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘/content/gdrive/My Drive/Colab_Models’: File exists


In [24]:
# Define the loss function
cross_entropy = keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    reduction='none',
)

# EarlyStopping criteria
early_stopping = keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Learning Rate Scheduler for the optimizer
class LRSchedule(keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, post_warmup_learning_rate, warmup_steps):
        super().__init__()
        self.post_warmup_learning_rate = post_warmup_learning_rate
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        global_step = tf.cast(step, tf.float32)
        warmup_steps = tf.cast(self.warmup_steps, tf.float32)
        warmup_progress = global_step / warmup_steps
        warmup_learning_rate = self.post_warmup_learning_rate * warmup_progress
        return tf.cond(
            global_step < warmup_steps,
            lambda: warmup_learning_rate,
            lambda: self.post_warmup_learning_rate,
        )


# Create a learning rate schedule
num_train_steps = len(train_dataset) * EPOCHS
num_warmup_steps = num_train_steps // 15
lr_schedule = LRSchedule(post_warmup_learning_rate=1e-4, warmup_steps=num_warmup_steps)

# Compile the model
caption_model.compile(optimizer=keras.optimizers.Adam(lr_schedule), loss=cross_entropy)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import _pickle as pickle
from keras_nlp.metrics import Bleu, Perplexity, RougeL
from keras.callbacks import Callback
import warnings
import logging
import tensorflow as tf
import json
import os

# # Set TensorFlow logging level to ERROR to suppress warnings and info messages
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')




class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.history = {'loss':[],'val_loss':[]}

    def on_batch_end(self, batch, logs={}):
        self.history['loss'].append(logs.get('loss'))
        with open(loss_model_path, "r+b") as myfile:
            myfile.write(pickle.dumps(self.history))

    def on_epoch_end(self, epoch, logs={}):
        self.history['val_loss'].append(logs.get('val_loss'))



class BleuScoreCallback(keras.callbacks.Callback):
    def __init__(self, validation_dataset, dim_batch, vectorization, index_lookup, max_decoded_sentence_length, n_batch=5):
        super().__init__()
        self.validation_dataset = validation_dataset
        self.dim_batch = dim_batch
        self.n_batch = n_batch
        self.vectorization = vectorization
        self.index_lookup = index_lookup
        self.max_decoded_sentence_length = max_decoded_sentence_length
        self.bleu_metric4 = Bleu( max_order=4)
        self.bleu_metric3 = Bleu( max_order=3)
        self.bleu_metric2 = Bleu( max_order=2)
        self.bleu_metric1 = Bleu( max_order=1)

    def on_epoch_end(self, epoch, logs=None):
        references = []
        predictions = []

        for batch_data in self.validation_dataset.take(self.n_batch):
            # reduce batch size if the number of samples is more than the dim_batch ( for limited memory devices)
            if len(batch_data[0]) > self.dim_batch:
                batch_data = (batch_data[0][:self.dim_batch], batch_data[1][:self.dim_batch])
            batch_img, batch_seq = batch_data
            img_embed = self.model.cnn_model(batch_img, training=False)
            encoded_img = self.model.encoder(img_embed, training=False)

            for i in range(batch_img.shape[0]):
                decoded_caption = "<start>"
                for j in range(self.max_decoded_sentence_length):
                    tokenized_caption = self.vectorization([decoded_caption])[:, :-1]
                    mask = tf.math.not_equal(tokenized_caption, 0)
                    predictions_batch = self.model.decoder(
                        tokenized_caption, encoded_img[i:i+1], training=False, mask=mask
                    )

                    if j < predictions_batch.shape[1]:
                        sampled_token_index = np.argmax(predictions_batch[0, j, :])
                        sampled_token = self.index_lookup[sampled_token_index]
                        if sampled_token == "<end>":
                            break
                        decoded_caption += " " + sampled_token
                    else:
                        break

                decoded_caption = decoded_caption.replace("<start> ", "").replace(" <end>", "").strip()
                pred_texts = decoded_caption

                # Convert ground truth token IDs to text ## TO fix that in this way the 5 caption are concatenated, we need to split them get the bleu score separately
                gt_texts = ' '.join([self.index_lookup[int(token)] for token in batch_seq[i, 1:].numpy().flatten() if int(token) != 0])


                references.append([gt_texts])
                predictions.append(pred_texts)

        bleu_score = self.bleu_metric1(references, predictions).numpy()
        print(f"Epoch {epoch+1} - BLEU1 Score: {bleu_score}")
        logs['BLEU-4'] = self.bleu_metric4(references, predictions).numpy()
        logs['BLEU-1'] = self.bleu_metric1(references, predictions).numpy()

class BleuScoreCallback(keras.callbacks.Callback):
    def __init__(self, validation_dataset, dim_batch, vectorization, index_lookup, max_decoded_sentence_length, n_batch=5):
        super().__init__()
        self.validation_dataset = validation_dataset
        self.dim_batch = dim_batch
        self.n_batch = n_batch
        self.vectorization = vectorization
        self.index_lookup = index_lookup
        self.max_decoded_sentence_length = max_decoded_sentence_length
        self.bleu_metric4 = Bleu(max_order=4)
        self.bleu_metric3 = Bleu(max_order=3)
        self.bleu_metric2 = Bleu(max_order=2)
        self.bleu_metric1 = Bleu(max_order=1)

    def on_epoch_end(self, epoch, logs=None):
        references = []
        predictions = []

        for batch_data in self.validation_dataset.take(self.n_batch):
            # Reduce batch size if the number of samples is more than the dim_batch (for limited memory devices)
            if len(batch_data[0]) > self.dim_batch:
                batch_data = (batch_data[0][:self.dim_batch], batch_data[1][:self.dim_batch])
            batch_img, batch_seq = batch_data
            img_embed = self.model.cnn_model(batch_img, training=False)
            encoded_img = self.model.encoder(img_embed, training=False)

            for i in range(batch_img.shape[0]):
                decoded_caption = "<start>"
                for j in range(self.max_decoded_sentence_length):
                    tokenized_caption = self.vectorization([decoded_caption])[:, :-1]
                    mask = tf.math.not_equal(tokenized_caption, 0)
                    predictions_batch = self.model.decoder(
                        tokenized_caption, encoded_img[i:i+1], training=False, mask=mask
                    )

                    if j < predictions_batch.shape[1]:
                        sampled_token_index = np.argmax(predictions_batch[0, j, :])
                        sampled_token = self.index_lookup[sampled_token_index]
                        if sampled_token == "<end>":
                            break
                        decoded_caption += " " + sampled_token
                    else:
                        break

                decoded_caption = decoded_caption.replace("<start> ", "").replace(" <end>", "").strip()
                pred_texts = decoded_caption

                # Convert ground truth token IDs to text and split them into individual captions
                gt_texts = [
                    ' '.join([self.index_lookup[int(token)] for token in caption if int(token) != 0])
                    for caption in batch_seq[i].numpy()
                ]

                references.extend([[gt_text] for gt_text in gt_texts])
                predictions.extend([pred_texts] * len(gt_texts))

        bleu_score1 = self.bleu_metric1(references, predictions).numpy()
        bleu_score4 = self.bleu_metric4(references, predictions).numpy()
        print(f"Epoch {epoch+1} - BLEU1 Score: {bleu_score1}, BLEU4 Score: {bleu_score4}")
        logs['BLEU-1'] = bleu_score1
        logs['BLEU-4'] = bleu_score4

class PerplexityCallback(Callback):
    def __init__(self, validation_dataset, dim_batch, vectorization, index_lookup, max_decoded_sentence_length, n_batch=5, verbose=False):
        super().__init__()
        self.validation_dataset = validation_dataset
        self.dim_batch = dim_batch
        self.n_batch = n_batch
        self.vectorization = vectorization
        self.index_lookup = index_lookup
        self.max_decoded_sentence_length = max_decoded_sentence_length
        self.perplexity_metric = Perplexity(from_logits=True)
        self.verbose = verbose

    def log(self, message):
        if self.verbose:
            print(message)

    def on_epoch_end(self, epoch, logs=None):
        total_perplexity = 0
        num_batches = 0

        for batch_data in self.validation_dataset.take(self.n_batch):
            if len(batch_data[0]) > self.dim_batch:
                batch_data = (batch_data[0][:self.dim_batch], batch_data[1][:self.dim_batch])
            batch_img, batch_seq = batch_data

            self.log(f"Batch sequence shape: {batch_seq.shape}")

            img_embed = self.model.cnn_model(batch_img, training=False)
            encoded_img = self.model.encoder(img_embed, training=False)

            for i in range(batch_img.shape[0]):  # For each image in the batch
                image_perplexities = []
                for c in range(batch_seq.shape[1]):  # For each caption of the image
                    tokenized_caption = self.vectorization(["<start>"])[:, :1]  # Take only the <start> token
                    predictions = []

                    self.log(f"Initial tokenized_caption shape: {tokenized_caption.shape}")

                    for j in range(1, min(self.max_decoded_sentence_length, batch_seq.shape[2] - 1)):
                        try:
                            mask = tf.math.not_equal(tokenized_caption, 0)
                            pred_batch = self.model.decoder(
                                tokenized_caption, encoded_img[i:i+1], training=False, mask=mask
                            )
                            predictions.append(pred_batch[:, -1:, :])  # Only keep the last prediction

                            next_token = tf.reshape(batch_seq[i, c, j], (1, 1))
                            self.log(f"Next token shape: {next_token.shape}, value: {next_token.numpy()}")
                            tokenized_caption = tf.concat([tokenized_caption, next_token], axis=1)
                            self.log(f"Updated tokenized_caption shape: {tokenized_caption.shape}")
                        except tf.errors.InvalidArgumentError as e:
                            self.log(f"Error at position {j}. Skipping this token.")
                            continue

                    if not predictions:
                        self.log("No valid predictions were made. Skipping this caption.")
                        continue

                    predictions = tf.concat(predictions, axis=1)
                    target = batch_seq[i:i+1, c, 1:j]  # Remove <start> token and limit to predicted length

                    self.log(f"Predictions shape: {predictions.shape}")
                    self.log(f"Target shape: {target.shape}")

                    # Ensure predictions and target have the same sequence length
                    min_len = min(predictions.shape[1], target.shape[1])
                    predictions = predictions[:, :min_len, :]
                    target = target[:, :min_len]

                    if min_len == 0:
                        self.log("No valid tokens to calculate perplexity. Skipping this caption.")
                        continue

                    try:
                        self.perplexity_metric.update_state(target, predictions)
                        caption_perplexity = self.perplexity_metric.result().numpy()
                        image_perplexities.append(caption_perplexity)
                    except tf.errors.InvalidArgumentError as e:
                        self.log(f"Error calculating perplexity: {e}")
                    finally:
                        self.perplexity_metric.reset_state()

                # Average perplexity across all captions for this image
                if image_perplexities:
                    avg_image_perplexity = sum(image_perplexities) / len(image_perplexities)
                    total_perplexity += avg_image_perplexity
                    num_batches += 1
                else:
                    self.log("No valid perplexities for this image. Skipping.")

        avg_perplexity = total_perplexity / num_batches if num_batches > 0 else 0
        self.log(f"Epoch {epoch+1} - Average Perplexity: {avg_perplexity:.4f}")
        logs['perplexity'] = avg_perplexity


class RougeScoreCallback(keras.callbacks.Callback):
    def __init__(self, validation_dataset, dim_batch, vectorization, index_lookup, max_decoded_sentence_length, n_batch=5, verbose=False):
        super().__init__()
        self.validation_dataset = validation_dataset
        self.dim_batch = dim_batch
        self.n_batch = n_batch
        self.vectorization = vectorization
        self.index_lookup = index_lookup
        self.max_decoded_sentence_length = max_decoded_sentence_length
        self.rouge_metric = RougeL()
        self.verbose = verbose

    def log(self, message):
        if self.verbose:
            print(message)

    def on_epoch_end(self, epoch, logs=None):
        references = []
        predictions = []

        for batch_data in self.validation_dataset.take(self.n_batch):
            # Reduce batch size if the number of samples is more than the dim_batch (for limited memory devices)
            if len(batch_data[0]) > self.dim_batch:
                batch_data = (batch_data[0][:self.dim_batch], batch_data[1][:self.dim_batch])
            batch_img, batch_seq = batch_data
            img_embed = self.model.cnn_model(batch_img, training=False)
            encoded_img = self.model.encoder(img_embed, training=False)

            for i in range(batch_img.shape[0]):
                decoded_caption = "<start>"
                for j in range(self.max_decoded_sentence_length):
                    tokenized_caption = self.vectorization([decoded_caption])[:, :-1]
                    mask = tf.math.not_equal(tokenized_caption, 0)
                    predictions_batch = self.model.decoder(
                        tokenized_caption, encoded_img[i:i+1], training=False, mask=mask
                    )

                    if j < predictions_batch.shape[1]:
                        sampled_token_index = np.argmax(predictions_batch[0, j, :])
                        sampled_token = self.index_lookup[sampled_token_index]
                        if sampled_token == "<end>":
                            break
                        decoded_caption += " " + sampled_token
                    else:
                        break

                decoded_caption = decoded_caption.replace("<start> ", "").replace(" <end>", "").strip()
                pred_texts = decoded_caption

                # Convert ground truth token IDs to text and split them into individual captions
                gt_texts = [
                    ' '.join([self.index_lookup[int(token)] for token in caption if int(token) != 0])
                    for caption in batch_seq[i].numpy()
                ]

                references.extend(gt_texts)
                predictions.extend([pred_texts] * len(gt_texts))

                if self.verbose:
                    for ref, pred in zip(gt_texts, [pred_texts] * len(gt_texts)):
                        print(f"Reference: {ref}")
                        print(f"Prediction: {pred}")

        rouge_scores = [self.rouge_metric(ref, pred) for ref, pred in zip(references, predictions)]
        rouge_precisions = [score['precision'].numpy() for score in rouge_scores]
        rouge_recalls = [score['recall'].numpy() for score in rouge_scores]
        rouge_f1_scores = [score['f1_score'].numpy() for score in rouge_scores]

        avg_rouge_precision = np.mean(rouge_precisions)
        avg_rouge_recall = np.mean(rouge_recalls)
        avg_rouge_f1_score = np.mean(rouge_f1_scores)

        print(f"Epoch {epoch+1} - ROUGE-L Precision: {avg_rouge_precision}, Recall: {avg_rouge_recall}, F1 Score: {avg_rouge_f1_score}")
        logs['ROUGE-L Precision'] = avg_rouge_precision
        logs['ROUGE-L Recall'] = avg_rouge_recall
        logs['ROUGE-L F1 Score'] = avg_rouge_f1_score


class SaveLogsCallback(keras.callbacks.Callback):
    def __init__(self, log_file_path):
        super().__init__()
        self.log_file_path = log_file_path
        self.logs = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['epoch'] = epoch + 1
        # Convert float32 values to regular floats
        logs = {k: (float(v) if isinstance(v, (np.float32, np.float64, tf.dtypes.DType)) else v) for k, v in logs.items()}
        self.logs.append(logs)

    def on_train_end(self, logs=None):
        with open(self.log_file_path, 'w') as f:
            json.dump(self.logs, f, indent=4)


# Define the path where you want to save the logs
log_file_name = 'test_training_logs.json'
if '/' not in loss_model_path:
    log_file_path = log_file_name
else:
    base_path = loss_model_path[:loss_model_path.rfind('/')]

    log_file_path = os.path.join(base_path, log_file_name)

print(log_file_path)
# Create an instance of the callback
save_logs_callback = SaveLogsCallback(log_file_path)


history = LossHistory()
vocab = vectorization.get_vocabulary()
index_lookup = dict(zip(range(len(vocab)), vocab))
bleu_callback = BleuScoreCallback(valid_dataset,10 ,vectorization, index_lookup, SEQ_LENGTH)

# Example usage
rouge_callback = RougeScoreCallback(
    validation_dataset=valid_dataset,
    dim_batch=10,
    vectorization=vectorization,
    index_lookup=index_lookup,
    max_decoded_sentence_length=SEQ_LENGTH,
    verbose=False
)

perplexity_callback = PerplexityCallback(
    validation_dataset=valid_dataset,
    dim_batch=10,
    vectorization=vectorization,
    index_lookup=index_lookup,
    max_decoded_sentence_length=SEQ_LENGTH
)

# Fit the model

caption_model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=valid_dataset,
    callbacks=[early_stopping, cp_callback, history, bleu_callback, perplexity_callback, rouge_callback, save_logs_callback],
)

/content/gdrive/My Drive/Colab_Models/test_training_logs.json
Epoch 1/15
259/259 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.2347 - loss: 28.3155
Epoch 1: saving model to /content/gdrive/My Drive/Colab_Models/cp_model_transformer_efficientnet_20240829_105032.weights.h5
Epoch 1 - BLEU1 Score: 0.15964056551456451, BLEU4 Score: 0.03013080544769764
Epoch 1 - ROUGE-L Precision: 0.35711169242858887, Recall: 0.2678021788597107, F1 Score: 0.3028911352157593
259/259 ━━━━━━━━━━━━━━━━━━━━ 1318s 5s/step - acc: 0.2351 - loss: 28.2891 - val_acc: 0.4232 - val_loss: 15.4409 - BLEU-1: 0.1596 - BLEU-4: 0.0301 - perplexity: 8688.7660 - ROUGE-L Precision: 0.3571 - ROUGE-L Recall: 0.2678 - ROUGE-L F1 Score: 0.3029 - epoch: 1.0000
Epoch 2/15
259/259 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.4152 - loss: 15.6116
Epoch 2: saving model to /content/gdrive/My Drive/Colab_Models/cp_model_transformer_efficientnet_20240829_105032.weights.h5
Epoch 2 - BLEU1 Score: 0.1728970855474472, BLEU4 Score: 0.03608785197138786
Epo

In [ ]:

# from keras_nlp.metrics import Bleu
# bleu = Bleu(max_order=3)
# print("Calculating BLEU Score es:" + str(bleu(["references text"], [" references test"]).numpy()))
# bleu(["references text"], [" references test"]).numpy()

In [ ]:
log_file_path = "test_training_logs.json"

In [ ]:
import json
import matplotlib.pyplot as plt
import math

# Load the JSON data from the log file
with open(log_file_path, 'r') as f:
    logs = json.load(f)

# Extract metrics
epochs = [log['epoch'] for log in logs]
metrics = {key: [log[key] for log in logs if key in log] for key in logs[0].keys() if key != 'epoch'}

# Determine the number of rows and columns for the grid
num_metrics = len(metrics)
num_cols = 3
num_rows = math.ceil(num_metrics / num_cols)

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))

# Flatten axes array for easy iteration
axes = axes.flatten()

# Plot each metric
for idx, (metric, values) in enumerate(metrics.items()):
    axes[idx].plot(epochs, values, label=metric)
    axes[idx].set_xlabel('Epoch')
    axes[idx].set_ylabel(metric)
    axes[idx].set_title(f'{metric} over epochs')
    axes[idx].legend()

# Remove any empty subplots
for idx in range(num_metrics, num_rows * num_cols):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

## Check sample predictions

In [ ]:
vocab = vectorization.get_vocabulary()
index_lookup = dict(zip(range(len(vocab)), vocab))
max_decoded_sentence_length = SEQ_LENGTH - 1
valid_images = list(valid_data.keys())


def generate_caption():
    # Select a random image from the validation dataset
    sample_img = np.random.choice(valid_images)

    # Read the image from the disk
    sample_img = decode_and_resize("coco/" + "train2014/" + sample_img)
    img = sample_img.numpy().clip(0, 255).astype(np.uint8)
    plt.imshow(img)
    plt.show()

    # Pass the image to the CNN
    img = tf.expand_dims(sample_img, 0)
    img = caption_model.cnn_model(img)

    # Pass the image features to the Transformer encoder
    encoded_img = caption_model.encoder(img, training=False)

    # Generate the caption using the Transformer decoder
    decoded_caption = "<start> "
    for i in range(max_decoded_sentence_length):
        tokenized_caption = vectorization([decoded_caption])[:, :-1]
        mask = tf.math.not_equal(tokenized_caption, 0)
        predictions = caption_model.decoder(
            tokenized_caption, encoded_img, training=False, mask=mask
        )
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = index_lookup[sampled_token_index]
        if sampled_token == "<end>":
            break
        decoded_caption += " " + sampled_token

    decoded_caption = decoded_caption.replace("<start> ", "")
    decoded_caption = decoded_caption.replace(" <end>", "").strip()
    print("Predicted Caption: ", decoded_caption)


# Check predictions for a few samples
generate_caption()
generate_caption()
generate_caption()

## End Notes

We saw that the model starts to generate reasonable captions after a few epochs. To keep
this example easily runnable, we have trained it with a few constraints, like a minimal
number of attention heads. To improve the predictions, you can try changing these training
settings and find a good model for your use case.